# テレオペ (WASD キーボード操作)

VNC 越しに Jupyter からキーボード (WASD) でロボットを操作します。

- W: 前進 / S: 後退 / A: 左旋回 / D: 右旋回
- キーを離すと停止
- セル 3 で明示的に停止 & 切断

In [ ]:
from pymodbus.client import ModbusSerialClient
import threading
import time

# --- 接続 ---
client = ModbusSerialClient(port="/dev/ttyAMA0", baudrate=115200, timeout=1)
client.connect()

result = client.read_holding_registers(0x00, count=2, device_id=1)
status = result.registers[0]
print(f"STATUS: IMU_OK={bool(status&1)}, Motor_OK={bool(status&2)}")

# --- 速度設定 ---
SPEED = 150       # mm/s 前進・後退
TURN_SPEED = 100  # mm/s 旋回

# --- ヘルパー ---
def to_u16(v):
    return v if v >= 0 else v + 0x10000

def send_speed(left, right):
    client.write_registers(0x40, values=[to_u16(int(left)), to_u16(int(right))], device_id=1)

def read_sensors():
    r = client.read_holding_registers(0x34, count=2, device_id=1)
    return r.registers if r and not r.isError() else None

# --- 現在の速度指令 (JS から更新される) ---
current_left = 0
current_right = 0

# --- バックグラウンドスレッド (200ms 間隔で送信) ---
running = True

def sender_loop():
    while running:
        send_speed(current_left, current_right)
        time.sleep(0.2)

sender_thread = threading.Thread(target=sender_loop, daemon=True)
sender_thread.start()
print("テレオペ準備完了 - 次のセルを実行してください")

In [ ]:
from IPython.display import HTML

HTML("""
<div id="teleop-hud" style="
    font-family: monospace; font-size: 18px;
    background: #1a1a2e; color: #0f0;
    padding: 20px; border-radius: 8px;
    text-align: center; user-select: none;
">
    <div style="margin-bottom: 10px; color: #aaa;">WASD で操作 / キーを離すと停止</div>
    <div id="teleop-key" style="font-size: 36px; margin: 15px 0;">---</div>
    <div id="teleop-cmd" style="font-size: 16px; color: #0ff;">L: 0  R: 0</div>
</div>

<script>
(function() {
    var SPEED = """ + str(SPEED) + """;
    var TURN_SPEED = """ + str(TURN_SPEED) + """;
    var activeKey = null;

    var keyMap = {
        'w': {label: 'W (前進)',  left:  SPEED,      right:  SPEED},
        's': {label: 'S (後退)',  left: -SPEED,      right: -SPEED},
        'a': {label: 'A (左旋回)', left: -TURN_SPEED, right:  TURN_SPEED},
        'd': {label: 'D (右旋回)', left:  TURN_SPEED, right: -TURN_SPEED}
    };

    function update(left, right, label) {
        document.getElementById('teleop-key').textContent = label;
        document.getElementById('teleop-cmd').textContent = 'L: ' + left + '  R: ' + right;
        var cmd = 'current_left = ' + left + '; current_right = ' + right;
        Jupyter.notebook.kernel.execute(cmd);
    }

    document.addEventListener('keydown', function(e) {
        var k = e.key.toLowerCase();
        if (keyMap[k] && activeKey !== k) {
            activeKey = k;
            var m = keyMap[k];
            update(m.left, m.right, m.label);
        }
    });

    document.addEventListener('keyup', function(e) {
        var k = e.key.toLowerCase();
        if (k === activeKey) {
            activeKey = null;
            update(0, 0, '---');
        }
    });
})();
</script>
""")

In [ ]:
# --- 停止 & 切断 ---
running = False
sender_thread.join(timeout=2)
send_speed(0, 0)
client.close()
print("停止 & 切断完了")